# I Hate Parking Tickets
John Eckberg

# TODO SECTION ON LISCENSES

Brian Dunn, The vice president of Duncan solutions (formerly the cities debt collector and current manager of parking IT infrastructure), put together, or was sent, a big excel sheet of all of the parking tickets given out in 2012. This was presumably created for the purposes of comparing pre-lpr (plate recognition) data to post-lpr data. The MDPW did not make a public statement offically anouncing the introduction of lpr's to their vehicles, but interviews place the onversion around early 2013. I found this data in 2021, ON A PUBLIC INTERNET FORUM.

In [1]:
#!pip3 install pandarallel --user

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

This is outdated data, but I'll have some fun with it anyways to inform how we should process our more recent ticket data, and get the scripts written.

The new data has been collected from an open records request, by me. While the data has been collected from an inpartial source, I delcare a conflict of interest bc I hate parking cops.

In [2]:
import pandas as pd

tickets = pd.read_csv('2012ticketdata.csv')

tickets['DATE'] = pd.to_datetime(tickets['ISSUEDATE'])
tickets = tickets.rename(columns={"LOCATIONDESC1" : "LOCATION", "ISSUENO":"ISSUENUM"})
tickets.set_index('DATE', inplace=True)


tickets.head()

,ISSUENUM,ISSUEDATE,ISSUETIME,VIODESCRIPTION,LOCATION
DATE,,,,,
2012-01-01,459215400,1/1/2012,2:43:00 PM,PARKING PROHIBITED BY OFFICIAL SIGN,5021 W CHAMBERS ST
2012-01-01,459145271,1/1/2012,3:13:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2342 N WEIL ST
2012-01-01,459145282,1/1/2012,3:14:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2354 N WEIL ST
2012-01-01,459215411,1/1/2012,3:39:00 PM,PARKED POSTED PRIVATE PROPERTY,7861 N 60TH ST
2012-01-01,459145363,1/1/2012,5:32:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2466 N 7TH ST


are you fucking kidding me? on windows any pandarallel function you call has to be self contatined with no imports. Steve Balmer ruined the internet.

In [3]:
from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 30, progress_bar=False)
# tickets["MKE"] = tickets["LOCATION"].apply(lambda x: str(x) + ", MILWAUKEE WI") 
# why tf do i have to typcast here?? says its AN INT?? I hate python

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,ISSUENUM,ISSUEDATE,ISSUETIME,VIODESCRIPTION,LOCATION
DATE,,,,,
2012-01-01,459215400,1/1/2012,2:43:00 PM,PARKING PROHIBITED BY OFFICIAL SIGN,5021 W CHAMBERS ST
2012-01-01,459145271,1/1/2012,3:13:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2342 N WEIL ST
2012-01-01,459145282,1/1/2012,3:14:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2354 N WEIL ST
2012-01-01,459215411,1/1/2012,3:39:00 PM,PARKED POSTED PRIVATE PROPERTY,7861 N 60TH ST
2012-01-01,459145363,1/1/2012,5:32:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2466 N 7TH ST


It is too much money to geocode 750k addresses. Instead, I downloaded the arcgis data for Milwaukee, and will create a dictionary mapping addresses their geometry from the shape file. link: https://gis-mclio.opendata.arcgis.com/datasets/MCLIO::address-points/about  theres a slight issue with using the arcgis data as a csv due to the weird x y format.

In [4]:
import geopandas as gpd

arcgis = gpd.read_file("Address_Points.zip")

In [5]:
arcgis.head()

,OBJECTID,Source_OID,Taxkey,HouseSx,Alt_ID,Dir,Street,StType,PDir,Muni,...,Y,DD_Lat,DD_Long,Addr_Statu,Street_Ln_,Bldg_Ply_O,Mailable,Source_GUI,HouseNo,geometry
0,1,1392,0220212001,NaN,NaN,N,PORT WASHINGTON,RD,NaN,Bayside,...,436107.0,NaN,NaN,Active,5192.0,170770.0,Y,08E4E13E-1222-4787-8481-73C5E99B82DC,8820.0,POINT (-87.91249 43.17744)
1,2,1409,0520006001,NaN,NaN,W,BROWN DEER,RD,NaN,Bayside,...,435714.0,NaN,NaN,Parking Lot,31030.0,NaN,Y,AE667850-606B-4B4D-BFB3-91F52ED3DF47,501.0,POINT (-87.91491 43.17641)
2,3,2325,0159979003,NaN,NaN,N,PORT WASHINGTON,RD,NaN,Bayside,...,439087.0,NaN,NaN,Active,20214.0,171832.0,Y,63617707-4CF9-4F5C-8D0A-7F36D72F9C5F,9289.0,POINT (-87.92072 43.18577)
3,4,5613,0189991002,NaN,NaN,N,LAKE,DR,NaN,Bayside,...,440109.0,NaN,NaN,Undeveloped,1501.0,NaN,N,99101094-B242-446D-928E-0B2C6597716A,9380.0,POINT (-87.89386 43.18808)
4,5,17033,0540263000,NaN,NaN,N,FIELDING,RD,NaN,Bayside,...,433786.0,NaN,NaN,Undeveloped,30276.0,NaN,N,591961CB-AB40-42BD-ADE8-2305B6D00293,8485.0,POINT (-87.89664 43.17078)


You see that geometry column? thats the juice. lets wrangle it quick. 

# note: you might be generating a lot of unmatched addresses by only taking the milwaukee group? or try updating arcgis file from website

In [6]:
# group by municipal, only take milwaukee.
# save file
munis = arcgis.groupby("Muni")
print(munis.groups.keys())
# drop all besides address and X & Y tuple
#milwaukee = munis.get_group("Milwaukee")



milwaukee = arcgis[['FullAddr','geometry']]
milwaukee.reset_index(drop=True, inplace=True)
# milwaukee['FullAddr'] = milwaukee['FullAddr'].apply(lambda x: str(x) + ", MILWAUKEE WI") 
# dont need to do this
milwaukee.to_csv("Milwaukee_PointData.csv")


milwaukee.head()

dict_keys(['Bayside', 'Brown Deer', 'Cudahy', 'Fox Point', 'Franklin', 'Glendale', 'Greendale', 'Greenfield', 'Hales Corners', 'Milwaukee', 'Oak Creek', 'River Hills', 'Shorewood', 'South Milwaukee', 'St. Francis', 'Wauwatosa', 'West Allis', 'West Milwaukee', 'Whitefish Bay'])


,FullAddr,geometry
0,8820 N PORT WASHINGTON RD,POINT (-87.91249 43.17744)
1,501 W BROWN DEER RD,POINT (-87.91491 43.17641)
2,9289 N PORT WASHINGTON RD,POINT (-87.92072 43.18577)
3,9380 N LAKE DR,POINT (-87.89386 43.18808)
4,8485 N FIELDING RD,POINT (-87.89664 43.17078)


To use this efficiently, ill convert into a dictionary & then use the key value pairs to map our ticket addresses to coordinates. Then we can convert the dataframe into a geopandas dataframe. string.hash_map runs based off of char value bit shifting not object id so this should work.

In [22]:
mke_dict = dict(zip(milwaukee['FullAddr'], milwaukee['geometry']))
missing_set = set()

In [23]:
print(len(mke_dict))

315500


In [24]:
#wrap this up into a class with class vars so it can just act as the Milwaukee address geocoder class.
def geocode_lat(geometry):
    if geometry in mke_dict:
        return mke_dict[geometry].x
    else:
        missing_set.add(geometry)
        return
    
def geocode_lng(geometry):
    if geometry in mke_dict:
        return mke_dict[geometry].y
    else:
        missing_set.add(geometry)
        return

This feels kinda hacky but its fast & works!

In [25]:
tickets["Lat"] = tickets["LOCATION"].parallel_apply(geocode_lat)
tickets["Lng"] = tickets["LOCATION"].parallel_apply(geocode_lng)
tickets = tickets.dropna()

Lets save that to a file to use from now on. On to the next notebook!

In [26]:
tickets.head(20)

,ISSUENUM,ISSUEDATE,ISSUETIME,VIODESCRIPTION,LOCATION,Lat,Lng
DATE,,,,,,,
2012-01-01,459145271,1/1/2012,3:13:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2342 N WEIL ST,-87.898738,43.061221
2012-01-01,459145282,1/1/2012,3:14:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2354 N WEIL ST,-87.898944,43.061436
2012-01-01,459215411,1/1/2012,3:39:00 PM,PARKED POSTED PRIVATE PROPERTY,7861 N 60TH ST,-87.986040,43.160480
2012-01-01,459145363,1/1/2012,5:32:00 PM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2466 N 7TH ST,-87.919640,43.063622
2012-01-01,458883670,1/1/2012,3:33:00 AM,PARKED LESS THAN 15 FEET FROM CROSSWALK,332 E SMITH ST,-87.905857,43.001334
2012-01-01,459257046,1/1/2012,2:57:00 AM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,2640 W HAYES AVE,-87.947676,43.001521
2012-01-01,459257050,1/1/2012,3:05:00 AM,PARKING PROHIBITED BY OFFICIAL SIGN,2263 S 23RD ST,-87.942695,43.003426
2012-01-01,458883692,1/1/2012,3:45:00 AM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,228 E SMITH ST,-87.907143,43.001321
2012-01-01,458883703,1/1/2012,3:50:00 AM,IMPROPERLY DISPLAYED VEHICLE REGISTRATION,228 E SMITH ST,-87.907143,43.001321


In [27]:
tickets.to_csv("Tickets2012LatLngDropNa.csv")

In [28]:
print(len(tickets.index))

338344


# So we lost like half our tickets in the geocoding proccess... This is after taking more than just the milwaukee group. This needs to be addressed. Have addresses in the city changed that much since 2012?